<a href="https://colab.research.google.com/github/karthikravi123/pytorch_practice/blob/main/Linear_Regresssion_model_with_pytorch_component.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.Data gathering  
2.Data preprocessing  
3.Feature Engineering  
4.Model Training (create own model instead of scikit learn)  
5.Testing  


In [62]:
##dataset used
#https://www.kaggle.com/datasets/mirichoi0218/insurance

In [63]:
!pip install kaggle

In [64]:
import kagglehub

path  = kagglehub.dataset_download("mirichoi0218/insurance")
path

Using Colab cache for faster access to the 'insurance' dataset.


'/kaggle/input/insurance'

In [65]:
os.listdir('/root/.cache/kagglehub/datasets/mirichoi0218/insurance/versions/1')


['insurance.csv']

In [66]:
import pandas as pd


In [67]:
df = pd.read_csv(os.path.join(path,'insurance.csv'))

In [68]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [69]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [70]:
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [72]:
import torch
import torch.nn as nn
##custom neural network
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split

In [73]:
#split dataset before encoding
train_df,test_df  = train_test_split(df,test_size=0.2,random_state=42)

In [74]:
##encode categorical variable
label_encoder = {}
for col in ['sex','smoker','region']:
  le = LabelEncoder()
  train_df[col] = le.fit_transform(train_df[col])
  test_df[col] = le.transform(test_df[col])
  label_encoder[col] = le



In [75]:
label_encoder

{'sex': LabelEncoder(), 'smoker': LabelEncoder(), 'region': LabelEncoder()}

In [76]:
##features and target

X_train = train_df.drop(columns=['charges'])
y_train = train_df['charges']

In [77]:

X_test = test_df.drop(columns=['charges'])
y_test = test_df['charges']

In [78]:
y_train.head()

,charges
560,9193.83850
1285,8534.67180
1142,27117.99378
969,8596.82780
486,12475.35130


In [79]:
#normalize features

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

pytorch model doesnt work with numpy pandas  with tensor
convert numpy pandas to tensor

In [80]:
##convert to tensors

X_train_tensor = torch.tensor(X_train,dtype = torch.float32)

y_train_tensor = torch.tensor(y_train.values,dtype = torch.float32).view(-1,1)
##change the view - flatten it

X_test_tensor = torch.tensor(X_test,dtype = torch.float32)
y_test_tensor = torch.tensor(y_test.values,dtype = torch.float32).view(-1,1)


.view(-1, 1)

What it does: Reshapes a PyTorch tensor.

-1 tells PyTorch: "figure out this dimension automatically based on the other dimension."

1 turns it into a column vector.

In [81]:
##Define neural network model

class SimpleNNRegresionModel(nn.Module):
  def __init__(self,input_dim):
    super(SimpleNNRegresionModel,self).__init__()
    self.network = nn.Sequential(
    nn.Linear(input_dim,64),
    nn.ReLU(),
    nn.Linear(64,128),
    nn.ReLU(),
    nn.Linear(128,1)
    )

  def forward(self,x):
    return self.network(x)


In [82]:
X_train_tensor.shape

torch.Size([1070, 6])

In [83]:
input_dim = X_train_tensor.shape[1]
model = SimpleNNRegresionModel(input_dim)

In [84]:
print(model)

SimpleNNRegresionModel(
  (network): Sequential(
    (0): Linear(in_features=6, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=1, bias=True)
  )
)


In [85]:
##loss and optimizer

#standard variable name used in loss function
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(),lr=0.01)


In [86]:
##training loop

epochs = 10000

for epoch in range(epochs):
  ##in model train - tells pytorch we eant to calculate gradients,store history  model training
  ##previous weights,what i s currrent weight ,pytorch tracks
  model.train()

  optimizer.zero_grad()
  prediction = model(X_train_tensor)
  loss = criterion(prediction,y_train_tensor)
  #calculate  gradients with loss back propogration
  loss.backward()

  #model.eval test model-pytorch dont want save or access it

  ##with optimizer update weights and gradients
  optimizer.step()

  if (epoch+10)%100 == 0 :
    print(f"epoch [{epoch+1}], loss:{loss.item():.4f}")

epoch [91], loss:55647036.0000
epoch [191], loss:32573348.0000
epoch [291], loss:30230720.0000
epoch [391], loss:28127624.0000
epoch [491], loss:26669020.0000
epoch [591], loss:25805348.0000
epoch [691], loss:25152648.0000
epoch [791], loss:24688842.0000
epoch [891], loss:24321086.0000
epoch [991], loss:24052716.0000
epoch [1091], loss:23831968.0000
epoch [1191], loss:23640072.0000
epoch [1291], loss:23473596.0000
epoch [1391], loss:23332980.0000
epoch [1491], loss:23217360.0000
epoch [1591], loss:23117440.0000
epoch [1691], loss:23030050.0000
epoch [1791], loss:22940874.0000
epoch [1891], loss:22858604.0000
epoch [1991], loss:22804970.0000
epoch [2091], loss:22752998.0000
epoch [2191], loss:22703866.0000
epoch [2291], loss:22652802.0000
epoch [2391], loss:22605600.0000
epoch [2491], loss:22559256.0000
epoch [2591], loss:22512588.0000
epoch [2691], loss:22484636.0000
epoch [2791], loss:22462682.0000
epoch [2891], loss:22433200.0000
epoch [2991], loss:22406162.0000
epoch [3091], loss:22

In [87]:
##model evaluation

model.eval()

##detach from memory and convert to numpy
y_pred = model(X_test_tensor).detach().numpy()

In [88]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
y_test_numpy = y_test_tensor.numpy()

##calcuate metrics
mse = mean_squared_error(y_test_numpy,y_pred)
rmse = mse ** 0.5
mae =mean_absolute_error(y_test_numpy,y_pred)
r2 = r2_score(y_test_numpy,y_pred)
print(f"mse {mse} \n , mae {mae},\n, r2 {r2}")


mse 36719064.0 
 , mae 3792.349853515625,
, r2 0.7634823322296143


In [96]:
def predict_charges(age,sex,bmi,children,smoker,region):
  input_data = pd.DataFrame([[age,sex,bmi,children,smoker,region]],
               columns= ['age','sex','bmi','children','smoker','region'])
    ##encode categorical variable

  for col in ['sex','smoker','region']:
    input_data[col] = label_encoder[col].transform(input_data[col])
  input_data= scaler.transform(input_data)
  input_tensor = torch.tensor(input_data,dtype=torch.float32)
  predict_charge = model(input_tensor).item()
  return predict_charge

In [98]:
predicted = predict_charges(age=30, sex='male', bmi=25.0, children=2, smoker='yes', region='southwest')
print(f"predicted charges: ${predicted:.2f}")


predicted charges: $18438.05
